## 1. Semantic search with all-MiniLM-L6-v2 embedding

In [2]:
from sentence_transformers import SentenceTransformer
import chromadb
import pandas as pd
from tqdm import tqdm
from rich import print
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
pd.options.mode.chained_assignment = None  # default='warn'

### Preprocessing and splitting

In [20]:
df = pd.read_csv('data/bigvul_devign_cvefixes_neuralsentry_commits.csv')
df = df.drop_duplicates(subset=['commit_hash'],keep='first')
df = df.dropna(subset=['commit_msg', 'commit_hash'])
train_df, test_df = train_test_split(df, test_size=0.2, random_state=420)

linux_test_df = pd.read_csv('data/linux_test.csv')
vulnfix = linux_test_df[linux_test_df['labels'] == 1]
non_vulnfix = linux_test_df[linux_test_df['labels'] == 0]
linux_test_df = pd.concat([non_vulnfix, vulnfix])
linux_test_df.shape, test_df.shape

((2330, 6), (6998, 8))

### Embed commit for train and test dataset

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

def generate_embeddings(commit):
  pbar.update()
  return model.encode(commit).tolist()


with tqdm(total=len(train_df), desc="Generate embedding for train dataset") as pbar:
  train_df['embedded_commit'] = train_df['commit_msg'].apply(lambda x: generate_embeddings(x))

Generate embedding for train dataset: 100%|██████████| 27992/27992 [27:09<00:00, 17.18it/s]  


In [5]:
with tqdm(total=len(test_df), desc="Generate embedding for test dataset") as pbar:
  test_df['embedded_commit'] = test_df['commit_msg'].apply(lambda x: generate_embeddings(x))

with tqdm(total=len(linux_test_df), desc="Generate embedding for linux test dataset") as pbar:
  linux_test_df['embedded_commit'] = linux_test_df['commit_msg'].apply(lambda x: generate_embeddings(x))

Generate embedding for linux test dataset: 100%|██████████| 2330/2330 [05:29<00:00,  7.08it/s]


### Feeding training data into ChromaDB

In [7]:
train_embeddings = train_df['embedded_commit']
train_labels = train_df['labels'].tolist()
train_hash = train_df['commit_hash']

client = chromadb.PersistentClient(path='db/all-MiniLM-L6-v2 embedding/')
collection = client.create_collection("all-MiniLM-L6-v2_embedding")

collection.add(
    embeddings=[embed for embed in train_embeddings],
    metadatas=[{"label": str(train_labels[i])} for i in range(len(train_df))],
    ids=[str(hashes) for hashes in train_hash]
)


### Prediction (BigVul, Devign, CVEfixes, NeuralSentry commits)

In [13]:
client = chromadb.PersistentClient(path='db/all-MiniLM-L6-v2 embedding/')
collection = client.get_collection(name="all-MiniLM-L6-v2_embedding")
y_pred = []
for index, row in test_df.iterrows():
    query_result = collection.query(
        query_embeddings=[ row['embedded_commit'] ],
        n_results=2,
    )
    y_pred.append(query_result['metadatas'][0][0]['label'])

print(query_result) # Last row of the test dataset
y_pred = [int(x) for x in y_pred]

{
    'ids': [['cdf1512ebac8b95afad3ec4352550f0f0240ce26', '993092dcd3fb7ec84fdd1cd3b28a3973740b22bf']],
    'distances': [[0.34146153926849365, 0.4003959894180298]],
    'metadatas': [[{'label': '0'}, {'label': '0'}]],
    'embeddings': None,
    'documents': [[None, None]]
}

### Generate Metrics for (BigVul, Devign, CVEfixes, NeuralSentry commits)

In [14]:
y_test = test_df['labels'].tolist()
report = classification_report(y_test,y_pred,target_names=["non-bugfix", "bugfix"],)
confusion_matrix_metric = confusion_matrix(y_test,y_pred)
accuracy_metric = accuracy_score(y_test, y_pred)
print(  "\n\n>>> Confusion Matrix:",
  f"\nTP: {confusion_matrix_metric[1][1]}, FP: {confusion_matrix_metric[0][1]}",
  f"\nFN: {confusion_matrix_metric[1][0]}, TN: {confusion_matrix_metric[0][0]}",
  "\n\n>>> Accuracy: ",
  accuracy_metric,
  "\n\n>>>: Classification Report:\n",
  report)

>>> Confusion Matrix: 
TP: 2798, FP: 919 
FN: 757, TN: 2524 

>>> Accuracy:  0.7605030008573879 

>>>: Classification Report:
               precision    recall  f1-score   support

  non-bugfix       0.77      0.73      0.75      3443
      bugfix       0.75      0.79      0.77      3555

    accuracy                           0.76      6998
   macro avg       0.76      0.76      0.76      6998
weighted avg       0.76      0.76      0.76      6998

### Prediction (Linux CVEs)

In [15]:
client = chromadb.PersistentClient(path='db/all-MiniLM-L6-v2 embedding/')
collection = client.get_collection(name="all-MiniLM-L6-v2_embedding")
y_pred = []
for index, row in linux_test_df.iterrows():
    query_result = collection.query(
        query_embeddings=[ row['embedded_commit'] ],
        n_results=2,
    )
    y_pred.append(query_result['metadatas'][0][0]['label'])

print(query_result) # Last row of the test dataset
y_pred = [int(x) for x in y_pred]

{
    'ids': [['9903e41ae1f5d50c93f268ca3304d4d7c64b9311', '822d15ded8887742ea7ea4ddbfcfebb443813dd3']],
    'distances': [[0.8187517523765564, 0.8839223384857178]],
    'metadatas': [[{'label': '1'}, {'label': '1'}]],
    'embeddings': None,
    'documents': [[None, None]]
}

### Generate Metrics for (Linux CVEs)

In [16]:
y_test = linux_test_df['labels'].tolist()
report = classification_report(y_test,y_pred,target_names=["non-bugfix", "bugfix"],)
confusion_matrix_metric = confusion_matrix(y_test,y_pred)
accuracy_metric = accuracy_score(y_test, y_pred)
print(  "\n\n>>> Confusion Matrix:",
  f"\nTP: {confusion_matrix_metric[1][1]}, FP: {confusion_matrix_metric[0][1]}",
  f"\nFN: {confusion_matrix_metric[1][0]}, TN: {confusion_matrix_metric[0][0]}",
  "\n\n>>> Accuracy: ",
  accuracy_metric,
  "\n\n>>>: Classification Report:\n",
  report)

>>> Confusion Matrix: 
TP: 1636, FP: 197 
FN: 191, TN: 306 

>>> Accuracy:  0.8334763948497854 

>>>: Classification Report:
               precision    recall  f1-score   support

  non-bugfix       0.62      0.61      0.61       503
      bugfix       0.89      0.90      0.89      1827

    accuracy                           0.83      2330
   macro avg       0.75      0.75      0.75      2330
weighted avg       0.83      0.83      0.83      2330